In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import signal

In [2]:
dic='/Users/shiwuzhang/WS/ASTRO/MAMMOTH_KCWI/analysis results/newresults/specmap/'
specmap_name='spec_ap.out'
ramap_name='ra_ap.out'
decmap_name='dec_ap.out'
spectral_axis_name='spec_axis.out'
specmap=np.genfromtxt(dic+specmap_name,delimiter=',')
ramap=np.genfromtxt(dic+ramap_name,delimiter=',')
decmap=np.genfromtxt(dic+decmap_name,delimiter=',')
spectral_axis=np.genfromtxt(dic+spectral_axis_name,delimiter=',')

In [3]:
spec_noise=[]
def signalfilter(data,N,wn):
    b,a=signal.butter(N,wn,'highpass')
    noise=signal.filtfilt(b,a,data)
    return noise

for i in range(25):
    noise=signalfilter(specmap[i]/specmap[i].max(),3,0.3)
    spec_noise.append(noise)
spec_noise=np.array(spec_noise)
spec_sigma=np.std(spec_noise,axis=1)

/usr/local/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [4]:
def gaussian(x,mu,sigma,A):
    
    exp_index=-0.5*np.power((x-mu)/sigma,2)
    y=A*np.exp(exp_index)
    return y

def emissionfit(fitfunc,x,y,initpara=None):
    popt,pcov=curve_fit(fitfunc,x,y,p0=initpara,
                        method='lm',maxfev=100000)
    return popt,pcov

In [5]:
spec_axis_model=np.linspace(spectral_axis.min(),spectral_axis.max(),500)
specmap_model=[]
para=[]
for i in range(25):
    popt,pcov=emissionfit(gaussian,spectral_axis,
                          specmap[i],[0,300,1])
    spec_model=gaussian(spec_axis_model,*popt)
    specmap_model.append(spec_model)
    para.append(popt)
para=np.array(para)
specmap_model=np.array(specmap_model)

In [6]:
dic='/Users/shiwuzhang/WS/ASTRO/MAMMOTH_KCWI/analysis results/newresults/specmap/'
np.savetxt(dic+'spec_ap_model.out',specmap_model,delimiter=',')
np.savetxt(dic+'spec_axis_model.out',spec_axis_model,delimiter=',')
np.savetxt(dic+'spec_axis_para.out',para,delimiter=',')
np.savetxt(dic+'spec_noise.out',spec_sigma,delimiter=',')